# Работа с гиперпараметрами моделей

В машинном обучении успешное построение модели зависит не только от выбора алгоритма, но и от правильной настройки его параметров. Чтобы модель показывала высокую точность и хорошо обобщала закономерности в данных, необходимо правильно настроить гиперпараметры.

**Подбор гиперпараметров** — это процесс поиска таких значений гиперпараметров, при которых модель достигает наилучших результатов на валидационных данных.

Для начала давайте разберемся в определениях:

**1. Параметры (parameters)**

- Это внутренние переменные, которые модель оценивает в процессе обучения на основе данных.
- Они подбираются автоматически методом оптимизации (например, градиентным спуском).
- Параметры определяют конкретную структуру модели после обучения.

**Примеры параметров:**
- В линейной регрессии: коэффициенты (w) и смещение (b) в уравнении регрессии.
- В логистической регрессии: коэффициенты (w) и смещение (b) в уравнении.
- В SVM: вектора опорных точек.
- В деревьях решений и случайных лесах: структура дерева (какие признаки используются для разбиения).

**2. Гиперпараметры (hyperparameters)**
- Это параметры, которые не обучаются автоматически — их нужно задавать вручную до начала обучения модели.
- Они управляют процессом обучения и сложностью модели.
- Подбор гиперпараметров осуществляется отдельно от обучения модели.

In [1]:
!pip install opendatasets --quiet

In [2]:
import opendatasets as od
import pandas

od.download("https://www.kaggle.com/competitions/electricity-consumption")
# {"username":"adele1997","key":"72a5b06391529c16b0e444311c9af408"}

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: adele1997
Your Kaggle Key: ··········
Extracting archive ./electricity-consumption/electricity-consumption.zip to ./electricity-consumption


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd

df = pd.read_csv("/content/electricity-consumption/train.csv")
df

,datetime,total
0,01.01.2005 00:00:00,NaN
1,01.01.2005 01:00:00,154139.8084
2,01.01.2005 02:00:00,157818.3593
3,01.01.2005 03:00:00,149310.6991
4,01.01.2005 04:00:00,138282.0380
...,...,...
35059,31.12.2008 19:00:00,249376.3608
35060,31.12.2008 20:00:00,246510.5725
35061,31.12.2008 21:00:00,226469.4133
35062,31.12.2008 22:00:00,199907.3942


## Возьмем наш Pipeline обработки с предыдущей лекции

In [5]:
import pandas as pd
import numpy as np
import math
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class TimeFeaturesExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self  # Мы не нуждаемся в обучении для этой трансформации

    def transform(self, X):
        # Извлекаем временные признаки
        X['year'] = X['datetime'].dt.year
        X['month'] = X['datetime'].dt.month
        X['season'] = X['datetime'].dt.quarter
        X['day'] = X['datetime'].dt.day
        X['hour'] = X['datetime'].dt.hour
        X['dayofyear'] = X['datetime'].dt.day_of_year
        X['dayofweek'] = X['datetime'].dt.day_of_week
        X['is_weekend'] = X['dayofweek'].isin([5, 6]).astype(int)

        # Убираем колонку 'datetime' после обработки
        X = X.drop(columns=['datetime'])

        return X

In [6]:
class TrigonometricFeaturesExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Добавляем тригонометрические признаки для временных признаков
        X['sin_month'] = X['month'].apply(math.sin)
        X['cos_month'] = X['month'].apply(math.cos)

        X['sin_hour'] = X['hour'].apply(math.sin)
        X['cos_hour'] = X['hour'].apply(math.cos)

        X['hour_sin'] = (X['hour'] / 23 * 2 * np.pi).apply(math.sin)
        X['hour_cos'] = (X['hour'] / 23 * 2 * np.pi).apply(math.cos)

        X['month_sin'] = ((X['month'] - 1) / 11 * 2 * np.pi).apply(math.sin)
        X['month_cos'] = ((X['month'] - 1) / 11 * 2 * np.pi).apply(math.cos)

        # Преобразуем дату в индекс, если нужно для дальнейшего использования
        day = 24
        year = 365.2425 * day

        X['day_sin'] = (X['hour'] * 2 * np.pi / day).apply(math.sin)
        X['day_cos'] = (X['hour'] * 2 * np.pi / day).apply(math.cos)

        X['year_sin'] = (X['hour'] * 2 * np.pi / year).apply(math.sin)
        X['year_cos'] = (X['hour'] * 2 * np.pi / year).apply(math.cos)

        return X

In [7]:
import pandas as pd

df = pd.read_csv("/content/electricity-consumption/train.csv")

# просто удаляем пропуски
df = df[~df['total'].isna()]
df['total'] = df['total'].astype('int64')

# Преобразуем строковый столбец 'datetime' в формат datetime
df['datetime'] = pd.to_datetime(df['datetime'], format="%d.%m.%Y %H:%M:%S")

# Разделяем данные на train и test, извлекая год прямо из 'datetime'
train = df[df['datetime'].dt.year != 2008]
test = df[df['datetime'].dt.year == 2008]

# Удалим ненужные колонки (например, целевую переменную) из обучающих данных
X_train = train.drop(columns=['total'])
y_train = train['total']

# Удалим ненужные колонки из тестовых данных
X_test = test.drop(columns=['total'])
y_test = test['total']

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# pipeline предобработки
pipeline_preprocessing = Pipeline([
    ('time_features', TimeFeaturesExtractor()), # призанки даты и времени
    ('trigonometric_features', TrigonometricFeaturesExtractor()), # Тригонометрические признаки
])

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Полный пайплайн с моделью
full_pipeline = Pipeline([
    ('preprocessing', pipeline_preprocessing),  # Готовый пайплайн для обработки данных
    ('scaler', StandardScaler()),               # Масштабирование данных
    ('model', RandomForestRegressor())          # Заглушка для модели
])

In [10]:
# Сетка гиперпараметров для Random Forest
param_grid = {
    'model__n_estimators': [50, 100, 150],
    'model__max_depth': [3, 5, 7],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

type(param_grid)

dict

## GridSearchCV

`GridSearchCV` — это метод из библиотеки `scikit-learn` для исчерпывающего перебора всех возможных комбинаций гиперпараметров, заданных в сетке (`grid`). Он автоматически проводит кросс-валидацию и выбирает лучшую комбинацию гиперпараметров на основе заданной метрики.

Как работает `GridSearchCV`:
1. Задаем сетку гиперпараметров — указываем, какие значения для гиперпараметров нужно протестировать.
2. Кросс-валидация — данные разбиваются на несколько блоков (fold'ов), и модель обучается на всех блоках, кроме одного, который используется для проверки. Этот процесс повторяется для всех комбинаций гиперпараметров.
3. Оценка качества — после каждой итерации фиксируется значение метрики на валидационном блоке.
4. Выбор лучшей модели — по итогам поиска выбирается комбинация гиперпараметров с лучшим значением метрики.

**Когда задаете сетку гиперпараметров, учитывайте следующее:**

- Гиперпараметры должны соответствовать модели, которую вы используете. Например, n_estimators используется в случайном лесе, но не в SVM.
- Чем больше вариантов в сетке, тем больше комбинаций и тем выше вычислительная сложность.


Общее количество комбинаций можно рассчитать по формуле:
$$\text{Кол-во комбинаций} = n_1 \times n_2 \times n_3 \times \ldots \times n_k$$
где $n_1, n_2, n_3, \ldots, n_k$ — количество вариантов значений для каждого гиперпараметра.
Для нашей сетки получится $3 * 3 * 2 * 2 = 36$ комбинаций гиперпараметров для случайного леса. Если используется кросс-валидация `cv=5`, то модель обучится $36 × 5 = 180$ раз.

Создаем объект `GridSearchCV`:
- `estimator` — модель, которую мы хотим настроить или `Pipeline`, который содержит модель.
- `param_grid` — сетка гиперпараметров.
- `cv` — количество фолдов для кросс-валидации.
- `scoring` — метрика для оценки качества модели.
- `n_jobs` — количество потоков для параллельных вычислений (-1 — использовать все).


In [11]:
%%time
# Подбор гиперпараметров через GridSearchCV
search = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      scoring='neg_mean_squared_error')

# Обучаем алгоритм подбора гиперпараметров
search.fit(X_train, y_train)

# Лучшие параметры
print(f"Best params: {search.best_params_}")
print(f"Best score: {search.best_score_}")
# учится примерно 15 минут!!!

Best params: {'model__max_depth': 7, 'model__min_samples_leaf': 2, 'model__min_samples_split': 5, 'model__n_estimators': 50}
Best score: -208695174.7074426
CPU times: user 11min 33s, sys: 2.78 s, total: 11min 36s
Wall time: 12min 38s


Атрибуты GridSearchCV:

*   `best_estimator_` — лучшая модель
*   `best_score_` — ошибка, полученная на лучшей модели.
*   `best_params_` — гиперпараметры лучшей модели


In [12]:
scores = pd.DataFrame(search.cv_results_)

print(scores.shape)

scores.head()

(36, 17)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__max_depth,param_model__min_samples_leaf,param_model__min_samples_split,param_model__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.262585,0.085170,0.048395,0.013020,3,1,2,50,"{'model__max_depth': 3, 'model__min_samples_le...",-4.385787e+08,-3.647334e+08,-2.740912e+08,-3.797816e+08,-4.111714e+08,-3.736713e+08,5.595114e+07,30
1,2.604109,0.563633,0.050751,0.010350,3,1,2,100,"{'model__max_depth': 3, 'model__min_samples_le...",-4.386594e+08,-3.646695e+08,-2.772841e+08,-3.802676e+08,-4.108987e+08,-3.743559e+08,5.481243e+07,35
2,3.873078,0.366763,0.061576,0.013210,3,1,2,150,"{'model__max_depth': 3, 'model__min_samples_le...",-4.378674e+08,-3.640170e+08,-2.756142e+08,-3.805813e+08,-4.098104e+08,-3.735781e+08,5.510584e+07,29
3,1.599628,0.540974,0.043893,0.011964,3,1,5,50,"{'model__max_depth': 3, 'model__min_samples_le...",-4.386203e+08,-3.627760e+08,-2.740881e+08,-3.776534e+08,-4.094950e+08,-3.725265e+08,5.576116e+07,25
4,2.957145,0.509831,0.064088,0.016950,3,1,5,100,"{'model__max_depth': 3, 'model__min_samples_le...",-4.384682e+08,-3.632932e+08,-2.758946e+08,-3.786171e+08,-4.102994e+08,-3.733145e+08,5.519431e+07,27


## Random Search (случайный поиск)

`RandomizedSearchCV` — это метод из библиотеки `scikit-learn` для случайного поиска наилучшей комбинации гиперпараметров. В отличие от `GridSearchCV`, где происходит полный перебор всех возможных комбинаций, `RandomizedSearchCV` выбирает случайное подмножество из сетки гиперпараметров и тестирует только его.


Как работает `RandomizedSearchCV`:
1. Задаем сетку гиперпараметров — так же, как в `GridSearchCV`, задаем диапазоны значений, которые хотим протестировать.
2. Случайный выбор гиперпараметров — случайным образом выбираются комбинации из заданной сетки.
3. Кросс-валидация — для каждой выбранной случайной комбинации гиперпараметров выполняется кросс-валидация.
4. Оценка и выбор лучшей комбинации — после всех прогонов выбирается лучшая комбинация по заданной метрике.


Создаем объект `RandomizedSearchCV`:
- `n_iter` — количество случайных комбинаций, которые мы хотим протестировать.
- `cv` — количество фолдов для кросс-валидации.
- `scoring` — метрика для оценки качества модели..
- `n_jobs` — количество потоков для параллельных вычислений.

In [13]:
# Сетка гиперпараметров для Random Forest
param_grid = {
    'model__n_estimators': [50, 100, 150],
    'model__max_depth': [3, 5, 7],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

type(param_grid)

dict

In [14]:
%%time
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# Определяем модель
# model = RandomForestRegressor()

# Создаем объект RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator = full_pipeline,
    param_distributions = param_grid,  # Диапазон параметров
    n_iter = 20,                       # 20 случайных комбинаций
    cv = 5,                            # 5-кратная кросс-валидация
    scoring = 'r2',                    # Метрика - точность
    n_jobs = -1,                       # Параллельное выполнение
    random_state = 42                  # Фиксируем сид для воспроизводимости
)

random_search.fit(X_train, y_train)

CPU times: user 8.51 s, sys: 568 ms, total: 9.08 s
Wall time: 5min 14s


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessing',
                                              Pipeline(steps=[('time_features',
                                                               TimeFeaturesExtractor()),
                                                              ('trigonometric_features',
                                                               TrigonometricFeaturesExtractor())])),
                                             ('scaler', StandardScaler()),
                                             ('model',
                                              RandomForestRegressor())]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'model__max_depth': [3, 5, 7],
                                        'model__min_samples_leaf': [1, 2],
                                        'model__min_samples_split': [2, 5],
                                        'model__n_estimators': [50, 100, 150]},
                   random_state=42, scoring='r2')

In [15]:
print(f"Best params: {random_search.best_params_}")
print(f"Best score: {random_search.best_score_}")

Best params: {'model__n_estimators': 100, 'model__min_samples_split': 2, 'model__min_samples_leaf': 2, 'model__max_depth': 7}
Best score: 0.8975547286842399


In [16]:
best_model = random_search.best_estimator_

In [17]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = best_model.predict(X_test)
print("Test MSE:", mean_squared_error(y_test, y_pred))
print("Test R2:", r2_score(y_test, y_pred))

Test MSE: 305118321.9683144
Test R2: 0.8314103861328884


In [29]:
test = pd.read_csv('/content/electricity-consumption/sample.csv')

# test = full_pipeline.transform(test)
test['datetime'] = pd.to_datetime(test['datetime'], format="%d.%m.%Y %H:%M:%S")

y_pred = best_model.predict(test.drop(columns=['total']))

In [23]:
test = pd.read_csv('/content/electricity-consumption/sample.csv')
test['total'] = y_pred

In [24]:
test.to_csv('submit.csv', index=False)

## Байесовская оптимизация (Bayesian Optimization)

**Байесовская оптимизация** — это продвинутый метод для поиска наилучшей комбинации гиперпараметров модели на основе построения вероятностной модели функции потерь (ошибки). Она использует информацию о предыдущих запусках для более эффективного поиска новых комбинаций гиперпараметров.

____

В отличие от `GridSearchCV` и `RandomizedSearchCV`, которые «слепо» перебирают параметры, байесовская оптимизация пытается умно предсказывать, какие гиперпараметры стоит протестировать, основываясь на предыдущих результатах.

____

Суть Байесовской оптимизации состоит в следующем:
1. Создается априорная модель функции потерь на основе начальных данных (случайных запусков). На основе этой модели строится аппроксимация функции потерь.
2. Далее используется стратегия `Exploration vs. Exploitation` для поиска следующих параметров:
    - `Exploration` — чтобы исследовать новые области.
    - `Exploitation` — чтобы углубляться в уже известные хорошие результаты.
3. После каждой итерации аппроксимация обновляется с учетом новых данных. Процесс повторяется до достижения критерия остановки.

Плюсы байесовской оптимизации:
- Учитывает результаты предыдущих запусков, следовательно осуществляет более эффективный поиск.
- Позволяет работать с непрерывными значениями гиперпараметров.
- Эффективно работает при сложных функциях потерь.
- Быстрее находит хорошее решение, чем полный перебор.

Минусы байесовской оптимизации:
- Не подходит для очень высокоразмерных пространств параметров.
- Требует больше ресурсов для построения аппроксимации.
- Не всегда находит глобальный минимум.


In [34]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 4.8 MB/s eta 0:00:00


In [38]:
%%time
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestRegressor

# Определяем пространство гиперпараметров
# необходимо добавить префикс "model__" к названиям гиперпараметров
param_space = {
    'model__n_estimators': (10, 200),
    'model__max_depth': (1, 20),
    'model__min_samples_split': (2, 10),
    'model__min_samples_leaf': (1, 5)
}

# Оптимизация с помощью Байесовского поиска
bayes_search = BayesSearchCV(
    estimator=full_pipeline,
    search_spaces=param_space,
    n_iter=20,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    random_state=42
)

# Обучение модели
bayes_search.fit(X_train, y_train)

CPU times: user 31.1 s, sys: 1.45 s, total: 32.5 s
Wall time: 14min 1s


BayesSearchCV(cv=5,
              estimator=Pipeline(steps=[('preprocessing',
                                         Pipeline(steps=[('time_features',
                                                          TimeFeaturesExtractor()),
                                                         ('trigonometric_features',
                                                          TrigonometricFeaturesExtractor())])),
                                        ('scaler', StandardScaler()),
                                        ('model', RandomForestRegressor())]),
              n_iter=20, n_jobs=-1, random_state=42, scoring='r2',
              search_spaces={'model__max_depth': (1, 20),
                             'model__min_samples_leaf': (1, 5),
                             'model__min_samples_split': (2, 10),
                             'model__n_estimators': (10, 200)})

In [39]:
# Вывод лучших параметров
print("Лучшие параметры:", bayes_search.best_params_)

Лучшие параметры: OrderedDict([('model__max_depth', 19), ('model__min_samples_leaf', 4), ('model__min_samples_split', 9), ('model__n_estimators', 89)])


In [40]:
best_bayes_model = bayes_search.best_estimator_
best_bayes_model

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('time_features', TimeFeaturesExtractor()),
                                 ('trigonometric_features',
                                  TrigonometricFeaturesExtractor())])),
                ('scaler', StandardScaler()),
                ('model',
                 RandomForestRegressor(max_depth=19, min_samples_leaf=4,
                                       min_samples_split=9, n_estimators=89))])

In [41]:
test = pd.read_csv('/content/electricity-consumption/sample.csv')

# test = full_pipeline.transform(test)
test['datetime'] = pd.to_datetime(test['datetime'], format="%d.%m.%Y %H:%M:%S")

y_pred = best_bayes_model.predict(test.drop(columns=['total']))

In [42]:
test = pd.read_csv('/content/electricity-consumption/sample.csv')
test['total'] = y_pred

test.to_csv('submit_bayes.csv', index=False)

## Выбор наилучшего алгоритм

In [43]:
import pandas as pd

df = pd.read_csv("/content/electricity-consumption/train.csv")

# просто удаляем пропуски
df = df[~df['total'].isna()]
df['total'] = df['total'].astype('int64')

# Преобразуем строковый столбец 'datetime' в формат datetime
df['datetime'] = pd.to_datetime(df['datetime'], format="%d.%m.%Y %H:%M:%S")

# Разделяем данные на train и test, извлекая год прямо из 'datetime'
train = df[df['datetime'].dt.year != 2008]
test = df[df['datetime'].dt.year == 2008]

# Удалим ненужные колонки (например, целевую переменную) из обучающих данных
X_train = train.drop(columns=['total'])
y_train = train['total']

# Удалим ненужные колонки из тестовых данных
X_test = test.drop(columns=['total'])
y_test = test['total']

In [45]:
%%time
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from skopt import BayesSearchCV

# Список моделей и их гиперпараметров
models = {
    "DecisionTree": (DecisionTreeRegressor(), {
        'model__max_depth': (1, 20),
        'model__min_samples_split': (2, 10)
    }),
    "RandomForest": (RandomForestRegressor(), {
        'model__n_estimators': (50, 100),
        'model__max_depth': (1, 20),
        'model__min_samples_split': (2, 10),
        'model__min_samples_leaf': (1, 5)
    }),
    "KNN": (KNeighborsRegressor(), {
        'model__n_neighbors': (1, 20),
        'model__weights': ['uniform', 'distance']
    }),
    # "LinearRegression": (LinearRegression(), {}),
    "SVR": (SVR(), {
        'model__C': (0.1, 100.0, 'log-uniform'),
        'model__epsilon': (0.01, 1.0, 'log-uniform'),
        'model__kernel': ['linear', 'rbf', 'poly']
    })
}

# Перебор моделей
best_model = None
best_score = -float("inf")
best_params = {}

for name, (model, params) in models.items():
    print(f"Обучаем {name}...")

    full_pipeline = Pipeline([
    ('preprocessing', pipeline_preprocessing),
    ('scaler', StandardScaler()),
    ('model', model)])

    # Байесовская оптимизация гиперпараметров
    bayes_search = BayesSearchCV(
        estimator=full_pipeline,
        search_spaces=params,
        n_iter=15,
        cv=5,
        scoring='r2',
        n_jobs=-1,
        random_state=42
    )

    # Обучение модели
    bayes_search.fit(X_train, y_train)

    # Проверка, стала ли эта модель лучшей
    if bayes_search.best_score_ > best_score:
        best_score = bayes_search.best_score_
        best_model = name
        best_params = bayes_search.best_params_

    print(f"{name} - R²: {bayes_search.best_score_:.4f}")

# Вывод лучшей модели
print("\nЛучшая модель:", best_model)
print("Лучшие параметры:", best_params)
print("Лучший R²:", best_score)

Обучаем DecisionTree...
DecisionTree - R²: 0.8998
Обучаем RandomForest...
RandomForest - R²: 0.9399
Обучаем KNN...
KNN - R²: 0.9417
Обучаем SVR...
SVR - R²: 0.7249

Лучшая модель: KNN
Лучшие параметры: OrderedDict([('model__n_neighbors', 4), ('model__weights', 'distance')])
Лучший R²: 0.9417203731332083
CPU times: user 1min 25s, sys: 3.8 s, total: 1min 29s
Wall time: 41min 6s
